In [1]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from itertools import combinations
from scipy.spatial.distance import cosine
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("data/English_evaluation.csv")

premise = df.Sentence1
hypothesis = df.Sentence2
label = df.gold_label
explanation = df.Generated_Explanation_EN


model_name = "roberta-large-mnli"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(model_name)

model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
print(device)


Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


cuda


In [2]:
premises = []
hypotheses = []
labels = []
explanations = []
predicted_labels = []
prediction_confidence = []
Faith_score = []
Density_score =[]
Token_recall = []


In [3]:
#masked predictions
def get_prob(premise,hypothesis,mask_positions=[]):
    inputs = tokenizer(premise,hypothesis, return_tensors="pt")

    if mask_positions:
        for pos in mask_positions:
            inputs['input_ids'][0][pos] = tokenizer.mask_token_id
            
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits,dim=1) #convert to prob
    return probs[0].cpu().numpy()

In [4]:
#masking 
def shap(premise,hypothesis, target_label):
     
    inputs = tokenizer(premise, hypothesis, return_tensors="pt")
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

    base_prob = get_prob(premise,hypothesis)[target_label]

    contributions = {}

    for i, token in enumerate(tokens):
        # Skip sPECIAL TOKENS 
        if token in ['<s>', '</s>', '<pad>']:
            contributions[i] = 0.0
            continue
    
        prob_without = get_prob(premise,hypothesis, mask_positions=[i])[target_label]

        contribution = base_prob - prob_without
        contributions[i] = contribution

        print(f"Token {i:2d} '{token:15s}': {contribution:+.4f}")

    return contributions,tokens    

In [5]:
for index, row in df.iterrows():
    premise = row['Sentence1']
    hypothesis = row['Sentence2']
    label = row['gold_label']
    explanation = row['Generated_Explanation_EN']

    premises.append(premise)
    hypotheses.append(hypothesis)
    labels.append(label)
    explanations.append(explanation)

    
    
    #base prediction no mask
    base_probs = get_prob(premise,hypothesis)
    predicted_label = np.argmax(base_probs)
    label_names = ["contradiction", "neutral", "entailment"]

    
    predicted_labels.append(predicted_label)
    prediction_confidence.append(base_probs[predicted_label])

    contrib_pred, tokens = shap(premise,hypothesis,predicted_label)
    
    #normalize
    total = sum(abs(v) for v in contrib_pred.values())
    contrib_pred_norm = {k: v/total for k, v in contrib_pred.items()}

    
    sorted_tokens = sorted(contrib_pred_norm.items(), key=lambda x: abs(x[1]), reverse=True)[:5]

    top_5 = []

    for idx, contrib in sorted_tokens:
        if contrib > 0:
            top_5.append((idx,contrib))
        if len(top_5) == 5:
            break
    contrib_sum = sum(contrib for idx,contrib in top_5)
    

    for idx, contrib in sorted_tokens:
    
        print(f"  {tokens[idx]:15s}: {contrib:+.4f}")

        
    expl_tokens = tokenizer.tokenize(explanation)
    print(expl_tokens)

    top_tokens_in_exp =[]
    for idx, contrib in top_5:
        token = tokens[idx]
        if token in expl_tokens:
            top_tokens_in_exp.append((token, contrib))

    for token, contrib in top_tokens_in_exp:
        print(f"  {token:15s}: {contrib:+.4f}")

    total_contrib_in_exp = sum(contrib for token, contrib in top_tokens_in_exp)

    if contrib_sum > 0:
        faithfulness_score = total_contrib_in_exp / contrib_sum
    else:
        faithfulness_score = 0.0

    Faith_score.append(faithfulness_score)

    token_recall = len(top_tokens_in_exp) / len(top_5) if len(top_5) > 0 else 0.0
    Token_recall.append(token_recall)

    density = total_contrib_in_exp / len(expl_tokens) if len(expl_tokens) >0 else 0.0
    Density_score.append(density)
    

# Save the Results
df_results = pd.DataFrame({
    "premise": premises,
    "hypothesis": hypotheses,
    "label": labels,
    "explanation": explanations,
    "predicted_label": predicted_labels,
    "prediction_confidence": prediction_confidence,
    "Faith_score": Faith_score,
    "Density_score": Density_score,
    "Token_recall": Token_recall
})

df_results.to_csv("data/Results.csv", index=False)
print("Done! CSV saved with results.")
    


Token  1 'A              ': +0.0001
Token  2 'Ġwoman         ': +0.0001
Token  3 'Ġin            ': +0.0001
Token  4 'Ġa             ': -0.0000
Token  5 'Ġblue          ': +0.6329
Token  6 'Ġa             ': +0.0001
Token  7 'pron           ': +0.0005
Token  8 'Ġcooking       ': +0.0000
Token  9 'Ġin            ': +0.0000
Token 10 'Ġa             ': +0.0001
Token 11 'Ġkitchen       ': -0.0001
Token 12 '.              ': -0.0001
Token 15 'A              ': +0.0001
Token 16 'Ġwoman         ': +0.0003
Token 17 'Ġin            ': +0.0002
Token 18 'Ġa             ': +0.0005
Token 19 'Ġpink          ': +0.6386
Token 20 'Ġa             ': -0.0000
Token 21 'pron           ': +0.0002
Token 22 'Ġsitting       ': +0.0001
Token 23 'Ġin            ': +0.0001
Token 24 'Ġa             ': -0.0001
Token 25 'Ġkitchen       ': -0.0000
Token 26 '.              ': -0.0001
  Ġpink          : +0.5011
  Ġblue          : +0.4966
  pron           : +0.0004
  Ġa             : +0.0004
  Ġwoman         : +0.0002
[

In [1]:
import pandas as pd

df_results = pd.read_csv("data/Results.csv")

# Show first few rows
print(df_results.head())

# General info
print(df_results.info())

# Summary statistics for numeric columns
print(df_results.describe())

metrics = ["prediction_confidence", "Faith_score", "Density_score", "Token_recall"]

for metric in metrics:
    print(f"\n--- {metric} ---")
    print(f"Mean: {df_results[metric].mean():.4f}")
    print(f"Median: {df_results[metric].median():.4f}")
    print(f"Min: {df_results[metric].min():.4f}")
    print(f"Max: {df_results[metric].max():.4f}")
    print(f"Standard Deviation: {df_results[metric].std():.4f}")

print("\nCorrelation between metrics:")
print(df_results[metrics].corr())

analysis = df_results[metrics].describe()
analysis.to_csv("data/Results_analysis.csv")
print("Saved statistical summary to Results_analysis.csv")



                                             premise  \
0      A woman in a blue apron cooking in a kitchen.   
1  This young man smiles as he discovers a horses...   
2  Three young boys playing with the mud on a beach.   
3          A lock box on a pole with the keys in it.   
4  A man pours water on plants in a vast field of...   

                                          hypothesis          label  \
0      A woman in a pink apron sitting in a kitchen.  contradiction   
1     The young man is at school taking a math test.  contradiction   
2                Three young girls are at the beach.  contradiction   
3  A blue lock box  attached to the side of the h...  contradiction   
4                                A man waters plants     entailment   

                                         explanation  predicted_label  \
0  The premise states the woman is wearing a blue...                0   
1  The premise describes the young man discoverin...                0   
2  The premise sp